In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Install necessary packages

In [ ]:
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers.git

Found existing installation: transformers 4.52.2
Uninstalling transformers-4.52.2:
  Successfully uninstalled transformers-4.52.2
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-usn16_81
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-usn16_81
  Resolved https://github.com/huggingface/transformers.git to commit e8b292e35f331d3c3de85f7e5d3496b0e13d3d6f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.53.0.dev0-py3-none-any.whl size=11382826 sha256=bc06a65277b0da5513b01d6af1c8210615d107a30d3ad0d328661bb873be257a
  Stored in directory: /tmp/pip-ephem-wheel-cache-0r5c3ikj/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers


In [ ]:
!pip install accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

### IDEFICS

#### 1st Try

In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM, Idefics2ForConditionalGeneration
import torch
from PIL import Image

# Model loading
model_id = "HuggingFaceM4/idefics2-8b"
token = "TOP SECRET"

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Processor & Model Load
processor = AutoProcessor.from_pretrained(model_id, token=token)
model = Idefics2ForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype = torch.float16,
    device_map = "auto",
    token = token
).eval()

model.safetensors.index.json:   0%|          | 0.00/74.4k [00:00<?, ?B/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.64G [00:00<?, ?B/s]

model-00002-of-00007.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00007.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

...

model-00006-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00005-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
image_path = "/content/drive/MyDrive/Apziva/4th_MonReader/monreader_images/images/testing/notflip/0001_000000002.jpg"
image = Image.open(image_path).convert("RGB")

# prompt
prompt = "<image>\nReturn only the exact visible text in the image."

# input using processor
inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)

In [ ]:
# Generate
outputs = model.generate(**inputs, max_new_tokens=100)

# Decoding
output_ids = model.generate(**inputs, max_new_tokens=100)
decoded = processor.tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("🧠 Raw Output:", decoded)

# 5. Extracting only text
import re
extracted_text = re.findall(r'"(.*?)"', decoded)
print("✅ Extracted Text:", extracted_text[0] if extracted_text else decoded)

🧠 Raw Output: 
Return only the exact visible text in the image.

 Networking.
✅ Extracted Text: 
Return only the exact visible text in the image.

 Networking.


#### 2nd Try

In [ ]:
from huggingface_hub import login
login("TOP_SECRET")

In [ ]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get('huggingface_pw')
login(token=hf_token)

In [ ]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM
from transformers.models.idefics2 import Idefics2ForConditionalGeneration

model_id = "HuggingFaceM4/idefics2-8b"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model and Process Load
processor = AutoProcessor.from_pretrained(model_id, token=True)
model = Idefics2ForConditionalGeneration.from_pretrained(
      model_id,
      device_map = "auto",
      torch_dtype = torch.float16,
      token=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Chat templates should be in a 'chat_template.jinja' file but found key='chat_template' in the processor's config. Make sure to move your template to its own file.


preprocessor_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

...

model-00001-of-00007.safetensors:   0%|          | 0.00/4.64G [00:00<?, ?B/s]

model-00005-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/4.25G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
# importing images
image_path = "/content/drive/MyDrive/Apziva/4th_MonReader/monreader_images/images/testing/notflip/0001_000000002.jpg"
image = Image.open(image_path).convert("RGB")

# prompt
prompt = "<image>\nExtract all visible text from the image. Only respond with the text."

# input using processor
inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)

# Generate
generate_ids = model.generate(**inputs, max_new_tokens=100)
output_text = processor.batch_decode(generate_ids, skip_special_tokens=True)[0]

print("🧠 IDEFICS Output:")
print(output_text)

🧠 IDEFICS Output:

Extract all visible text from the image. Only respond with the text.
 NETWORK ASIA.
